In [53]:
using LinearAlgebra

In [118]:
logical_zero = [0 for n in 1:2^7]
logical_zero[0b0000000+1] = 1
logical_zero[0b1111000+1] = 1
logical_zero[0b0110110+1] = 1
logical_zero[0b0011011+1] = 1
logical_zero[0b1001110+1] = 1
logical_zero[0b0101101+1] = 1
logical_zero[0b1100011+1] = 1
logical_zero[0b1010101+1] = 1
logical_zero = logical_zero/sqrt(8)

128-element Array{Float64,1}:
 0.35355339059327373
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.35355339059327373
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [119]:
logical_one = [0 for n in 1:2^7]
logical_one[0b1111111 - 0b0000000+1] = 1
logical_one[0b1111111 - 0b1111000+1] = 1
logical_one[0b1111111 - 0b0110110+1] = 1
logical_one[0b1111111 - 0b0011011+1] = 1
logical_one[0b1111111 - 0b1001110+1] = 1
logical_one[0b1111111 - 0b0101101+1] = 1
logical_one[0b1111111 - 0b1100011+1] = 1
logical_one[0b1111111 - 0b1010101+1] = 1
logical_one = logical_one/sqrt(8)

0.9999999999999999

In [56]:
function make_operator(op_order)
    op = 1
    for oo in op_order
        op = kron(op,oo)
    end
    return op
end

make_operator (generic function with 1 method)

In [57]:
# [X, X, X, X, I, I, I]
function make_pauli_operator(op_order)
    X = [0 1; 1 0]
    Y = [0 -1im; 1im 0]
    Z =[1 0; 0 -1]
    id = Matrix{Int}(I, 2, 2)
    
    paulis = Dict('I' => id, 'X' => X, 'Y' => Y, 'Z' => Z)
    
    new_op_order = [paulis[op] for op in op_order]
    return make_operator(new_op_order)
end

make_pauli_operator (generic function with 1 method)

In [109]:
function s(n, Q)
    if n == 1
        op_order = [ Q,   Q,  Q,  Q, 'I', 'I', 'I']
    elseif n == 2
        op_order = ['I',  Q,  Q, 'I', Q,   Q,  'I']
    elseif n == 3
        op_order = ['I', 'I', Q,  Q, 'I',  Q,   Q]
    end 
    
    return make_pauli_operator(op_order)
end

s (generic function with 1 method)

In [110]:
s1z = s(1, 'Z'); s1x = s(1, 'X');
s2z = s(2, 'Z'); s2x = s(2, 'X');
s3z = s(3, 'Z'); s3x = s(3, 'X');

In [163]:
function m(op, state) # measure <state|op|state>
    return op * state
end

function syndrome(error)
    return [s1x * error == error * s1x ? 1 : -1, 
            s2x * error == error * s2x ? 1 : -1, 
            s3x * error == error * s3x ? 1 : -1, 
            s1z * error == error * s1z ? 1 : -1, 
            s2z * error == error * s2z ? 1 : -1, 
            s3z * error == error * s3z ? 1 : -1]
end

syndrome (generic function with 2 methods)

In [164]:
logical_X = make_pauli_operator(['X','X','X','X','X','X','X']);
logical_Z = make_pauli_operator(['Z','Z','Z','Z','Z','Z','Z']);

In [165]:
println("X|0> =  |1> is ", logical_X * logical_zero == logical_one)
println("X|1> =  |0> is ", logical_X * logical_one == logical_zero)
println("Z|0> =  |0> is ", logical_Z * logical_zero == logical_zero)
println("Z|1> = -|1> is ", logical_Z * logical_one == -logical_one)

X|0> =  |1> is true
X|1> =  |0> is true
Z|0> =  |0> is true
Z|1> = -|1> is true


In [180]:
identity_arr = ['I', 'I', 'I', 'I', 'I', 'I', 'I']
minimum_error = Dict{String, String}()
for i in 1:length(identity_arr)
    E = copy(identity_arr)
    E[i] = 'X'
    E = make_pauli_operator(E)
    syn = syndrome(E)
    minimum_error[join(syn)] = "X$(i)"
    println("X$(i) = $(syn)")
end
for i in 1:length(identity_arr)
    E = copy(identity_arr)
    E[i] = 'Z'
    E = make_pauli_operator(E)
    syn = syndrome(E)
    minimum_error[join(syn)] = "Z$(i)"
    println("Z$(i) = $(syn)")
end
for i in 1:length(identity_arr)
    E = copy(identity_arr)
    E[i] = 'Y'
    E = make_pauli_operator(E)
    syn = syndrome(E)
    minimum_error[join(syn)] = "Y$(i)"
    println("Y$(i) = $(syn)")
end

X1 = [1, 1, 1, -1, 1, 1]
X2 = [1, 1, 1, -1, -1, 1]
X3 = [1, 1, 1, -1, -1, -1]
X4 = [1, 1, 1, -1, 1, -1]
X5 = [1, 1, 1, 1, -1, 1]
X6 = [1, 1, 1, 1, -1, -1]
X7 = [1, 1, 1, 1, 1, -1]
Z1 = [-1, 1, 1, 1, 1, 1]
Z2 = [-1, -1, 1, 1, 1, 1]
Z3 = [-1, -1, -1, 1, 1, 1]
Z4 = [-1, 1, -1, 1, 1, 1]
Z5 = [1, -1, 1, 1, 1, 1]
Z6 = [1, -1, -1, 1, 1, 1]
Z7 = [1, 1, -1, 1, 1, 1]
Y1 = [-1, 1, 1, -1, 1, 1]
Y2 = [-1, -1, 1, -1, -1, 1]
Y3 = [-1, -1, -1, -1, -1, -1]
Y4 = [-1, 1, -1, -1, 1, -1]
Y5 = [1, -1, 1, 1, -1, 1]
Y6 = [1, -1, -1, 1, -1, -1]
Y7 = [1, 1, -1, 1, 1, -1]


In [204]:
Y1     = make_pauli_operator(['Y', 'I', 'I', 'I', 'I', 'I', 'I'])
X1Z2   = make_pauli_operator(['X', 'Z', 'I', 'I', 'I', 'I', 'I'])
Y1Z2   = make_pauli_operator(['Y', 'Z', 'I', 'I', 'I', 'I', 'I'])
X1X2X3 = make_pauli_operator(['X', 'X', 'X', 'I', 'I', 'I', 'I'])
X1X2   = make_pauli_operator(['X', 'X', 'I', 'I', 'I', 'I', 'I'])
Z1Z2   = make_pauli_operator(['Z', 'Z', 'I', 'I', 'I', 'I', 'I'])

println("Y1     => $(syndrome(Y1    )) => $(get(minimum_error, join(syndrome(Y1    )), '-'))")
println("X1Z2   => $(syndrome(X1Z2  )) => $(get(minimum_error, join(syndrome(X1Z2  )), '-'))")
println("Y1Z2   => $(syndrome(Y1Z2  )) => $(get(minimum_error, join(syndrome(Y1Z2  )), '-'))")
println("X1X2X3 => $(syndrome(X1X2X3)) => $(get(minimum_error, join(syndrome(X1X2X3)), '-'))")
println("X1X2   => $(syndrome(X1X2  )) => $(get(minimum_error, join(syndrome(X1X2  )), '-'))")
println("Z1Z2   => $(syndrome(Z1Z2  )) => $(get(minimum_error, join(syndrome(Z1Z2  )), '-'))")


Y1     => [-1, 1, 1, -1, 1, 1] => Y1
X1Z2   => [-1, -1, 1, -1, 1, 1] => -
Y1Z2   => [1, -1, 1, -1, 1, 1] => -
X1X2X3 => [1, 1, 1, -1, 1, -1] => X4
X1X2   => [1, 1, 1, 1, -1, 1] => X5
Z1Z2   => [1, -1, 1, 1, 1, 1] => Z5
